<a href="https://colab.research.google.com/github/kassymoto/Kakaku/blob/master/kakaku_rank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf-8 -*-

"""
価格コムのあるカテゴリのランキングのURLを指定すると、
そのランキングの上位N個（多分40個まで）の価格推移のグラフを描画する。
"""

#もろもろのインポート
import requests
from bs4 import BeautifulSoup
from datetime import datetime

#データを格納するためのクラス
class Kakaku:
  url = ""
  price = []
  date = []
  title = ""

#上位何個を表示するか
top = 5

#ランキングのURL
rank_url = requests.get('http://kakaku.com/pc/pc-memory/ranking_0520/spec=101-6/')
soup = BeautifulSoup(rank_url.content, "html.parser")

#ランキングのページから各商品のURLを引っこ抜く
urls = []
num = 0
for i in soup.find_all("a", class_="withIcnLimited"):
  #価格推移グラフのURLは商品のURLにpricehistoryを追加したものである。
    urls.append(str(i.get("href")) + "pricehistory/")
    num+=1
    #指定した個数になったら終了する。
    if num==top:
      break
      
#上記ループで取得したURLは文字列なので、これを使ってrequestsする
url = []
for i in urls:
    url.append(requests.get(i))

#URLをインスタンスに登録していく
kakaku_list = []
for i in url:
  kakaku_tmp = Kakaku()
  kakaku_tmp.url = i
  kakaku_list.append(kakaku_tmp)

rank = 1
for kakaku in kakaku_list:
    soup = BeautifulSoup(kakaku.url.content, "html.parser")
    
    date = soup.find_all("td", class_="alignL")
    price = soup.find_all("td", class_="alignR itemviewColor06")
    
    title = soup.find("title").string.split(" ")[2:-1:]
    kakaku.title = str(rank) + ": " + str(" ".join(title))
    rank+=1
    
    date_tmp = []
    price_tmp = []
    
    for i,j in zip(date[0::2],price):
        if(str(i.string)=="現在"):
            date_tmp.append(datetime.now().strftime('%Y-%m-%d'))
        else:
            date_tmp.append("".join(str(i.string).split(" ")[:-1:]).replace("年","-").replace("月","-").replace("日",""))
        price_tmp.append(int(str(j.string[1::]).replace(',','')))
    
    kakaku.date = date_tmp
    kakaku.price = price_tmp

""" 
for i in kakaku_list:
    print(i.title)
    for j,k in zip(i.date,i.price):
        print(j,k)
    print()
"""

import pandas as pd
kakaku_Title = []
kakaku_Date = []
kakaku_Price = []

for i in kakaku_list:
    for j in range(len(i.date)):
        kakaku_Title.append(i.title)
    kakaku_Date += i.date
    kakaku_Price += i.price

kakaku_data = pd.DataFrame(
        {"Title":kakaku_Title,
         "Date":kakaku_Date,
         "Price":kakaku_Price})


import altair as alt

alt.Chart(kakaku_data, height=500 ,width=700).configure_legend(labelLimit=0).mark_line().encode(
        x="Date:T",
        y="Price:Q",
        color=alt.Color('Title', legend=alt.Legend(title="Title", orient='right'))
        )

Chart({
  config: Config({
    legend: LegendConfig({
      labelLimit: 0
    })
  }),
  data:            Date  Price                                              Title
  0    2019-04-01  11022  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  1     2019-3-26  11022  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  2     2019-3-26  10498  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  3     2019-3-26  11022  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  4     2019-3-25  10498  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  5     2019-3-23  11022  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  6     2019-3-22  11439  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  7     2019-3-22  11440  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  8     2019-3-21  11544  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  9     2019-3-21  11545  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  10    2019-3-20  11648  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  11    2019-3-20  11649  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  12    2019-3-19  11650  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  13    2019-3-19  11651  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  14    2019-3-18  11655  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  15    2019-3-18  11656  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  16    2019-3-16  11761  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  17    2019-3-15  11762  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  18    2019-3-15  11761  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  19    2019-3-15  11762  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  20    2019-3-14  11866  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  21    2019-3-13  11867  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  22    2019-3-12  11762  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  23    2019-3-12  11761  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  24    2019-3-10  11762  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  25    2019-3-10  11868  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  26    2019-3-10  11869  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  27    2019-3-10  11870  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  28     2019-3-9  11871  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  29     2019-3-9  11872  1: Corsair CMK16GX4M2A2666C16 [DDR4 PC4-21300 ...
  ..          ...    ...                                                ...
  140   2019-3-11   9690  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  141   2019-3-10   9698  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  142    2019-3-8  10960  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  143    2019-3-7  10970  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  144    2019-3-2  10980  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  145    2019-3-1  11413  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  146   2019-2-24  10980  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  147   2019-2-15  10980  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  148   2019-2-15  10990  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  149    2019-2-7  11470  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  150    2019-2-7  11480  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  151    2019-2-3  11480  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  152    2019-2-3  10778  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  153   2019-1-31  11480  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  154   2019-1-28  12370  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  155   2019-1-24  12380  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  156   2019-1-23  12920  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  157   2019-1-22  12920  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  158   2019-1-21  12930  4: G.Skill F4-2666C19D-16GNT [DDR4 PC4-21300 8...
  159   2019-1-18  12940  4: G.